In [5]:
import os
import geopandas as gpd
from shapely.geometry import Point, box as shapely_box
import simplekml
import numpy as np
import zipfile
import shutil

def make_shapefile(string, coords, width_km=5, height_km=5, output_format='GeoJSON'):

    # Approximate conversions
    km_per_degree_lat = 111  # Kilometers per degree of latitude
    km_per_degree_lon = 111 * np.cos(np.radians(coords[1]))  # Kilometers per degree of longitude at given latitude

    # Convert width and height from kilometers to degrees
    width = width_km / km_per_degree_lon
    height = height_km / km_per_degree_lat
    
    # Create folders for output if they don't exist
    output_folders = ['../data_store/data/shapefiles', './shapefiles']
    for output_folder in output_folders:
        os.makedirs(output_folder, exist_ok=True)

    # Create a GeoDataFrame with a point and set the CRS explicitly
    point = gpd.GeoDataFrame(geometry=[Point(coords)], crs="EPSG:4326")

    # Create a square (box) with a given width and height
    bounding_box = shapely_box(
        point.geometry.x[0] - width, 
        point.geometry.y[0] - height,
        point.geometry.x[0] + width, 
        point.geometry.y[0] + height
    )

    # Convert the box to a GeoDataFrame
    box_gdf = gpd.GeoDataFrame(geometry=[bounding_box], crs="EPSG:4326")

    # Save the file in the specified format
    for output_folder in output_folders:
        if output_format.lower() == 'geojson':
            filename = os.path.join(output_folder, f"{string}.geojson")
            box_gdf.to_file(filename, driver='GeoJSON')
            print(f"GeoJSON file '{filename}' created.")
        
        elif output_format.lower() == 'shapefile':
            # Create a temporary folder for shapefile components
            shapefile_folder = os.path.join(output_folder, string)
            os.makedirs(shapefile_folder, exist_ok=True)
            
            # Define the path for the shapefile components
            filename = os.path.join(shapefile_folder, f"{string}.shp")
            box_gdf.to_file(filename)
            
            # Zip the shapefile components
            zip_filename = os.path.join(output_folder, f"{string}.zip")
            with zipfile.ZipFile(zip_filename, 'w') as zipf:
                for file in os.listdir(shapefile_folder):
                    zipf.write(os.path.join(shapefile_folder, file), arcname=file)
            
            # Remove the temporary shapefile folder
            shutil.rmtree(shapefile_folder)
            print(f"Shapefile '{zip_filename}' created and zipped.")
        
        elif output_format.lower() == 'kml':
            # Create a KML object
            kml = simplekml.Kml()
            
            # Create a polygon from the box geometry
            polygon_coords = [(x, y) for x, y in box_gdf.geometry.apply(lambda geom: list(geom.exterior.coords))[0]]
            kml.newpolygon(name=f"Box around {string}", outerboundaryis=polygon_coords)
            
            filename = os.path.join(output_folder, f"{string}.kml")
            kml.save(filename)
            print(f"KML file '{filename}' created.")
        
        else:
            print(f"Unsupported format: {output_format}")
    
    return


In [6]:
import numpy as np

make_shapefile(string = "oregon_5",
               coords = (-121.606, 44.3238),
               width_km = 5,
               height_km = 5,
               output_format="kml")

KML file '../data_store/data/shapefiles/oregon_5.kml' created.
KML file './shapefiles/oregon_5.kml' created.


# No file generation, just corners

In [1]:
import numpy as np

def get_bounding_box_corners(coords, width_km=25, height_km=25):
    # Approximate conversions
    km_per_degree_lat = 111  # Kilometers per degree of latitude
    km_per_degree_lon = 111 * np.cos(np.radians(coords[1]))  # Kilometers per degree of longitude at given latitude

    # Convert width and height from kilometers to degrees
    width_deg = width_km / km_per_degree_lon
    height_deg = height_km / km_per_degree_lat

    # Calculate the southwest corner
    sw_lon = coords[0] - width_deg
    sw_lat = coords[1] - height_deg

    # Calculate the northeast corner
    ne_lon = coords[0] + width_deg
    ne_lat = coords[1] + height_deg

    # Return the corners as tuples
    return (sw_lat, sw_lon), (ne_lat, ne_lon)

In [2]:
# Example usage
sw_corner, ne_corner = get_bounding_box_corners((-93.4541, 47.5049),
                                                width_km=5,
                                                height_km=5)
print(f"Southwest corner: {sw_corner}")
print(f"Northeast corner: {ne_corner}")


Southwest corner: (47.459854954954956, -93.5207813241899)
Northeast corner: (47.54994504504504, -93.3874186758101)


In [23]:
# import numpy as np
# import simplekml

# def get_bounding_box_kml(coords, width_km=25, height_km=25, filename='bounding_box.kml'):
#     # Approximate conversions
#     km_per_degree_lat = 111  # Kilometers per degree of latitude
#     km_per_degree_lon = 111 * np.cos(np.radians(coords[1]))  # Kilometers per degree of longitude at given latitude

#     # Convert width and height from kilometers to degrees
#     width_deg = width_km / km_per_degree_lon
#     height_deg = height_km / km_per_degree_lat

#     # Calculate the southwest and northeast corners
#     sw_lon = coords[0] - width_deg
#     sw_lat = coords[1] - height_deg
#     ne_lon = coords[0] + width_deg
#     ne_lat = coords[1] + height_deg

#     # Calculate the other corners (northwest and southeast)
#     nw_lon, nw_lat = sw_lon, ne_lat
#     se_lon, se_lat = ne_lon, sw_lat

#     # Create a KML object
#     kml = simplekml.Kml()

#     # Define the coordinates for the bounding box in KML format (closed polygon)
#     bounding_box_coords = [(sw_lon, sw_lat), (nw_lon, nw_lat), (ne_lon, ne_lat), (se_lon, se_lat), (sw_lon, sw_lat)]

#     # Create a polygon from the bounding box coordinates
#     kml.newpolygon(name="Bounding Box", outerboundaryis=bounding_box_coords)

#     # Save the KML file
#     kml.save(filename)
#     print(f"KML file '{filename}' created.")

# # Example usage
# get_bounding_box_kml((26.634154, 67.36183), width_km=5, height_km=5, filename='../data_store/data/shapefiles/sodankyla_test.kml')


KML file '../data_store/data/shapefiles/sodankyla_test.kml' created.
